<a href="https://colab.research.google.com/github/benblunt/benblunt/blob/main%2Fbenbluntpublicprofile31/Intro_to_LLM_Monitoring_LangKit_and_WhyLabs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monitoring Large Language Models (LLMs) with LangKit

In this example we'll show how to generate out-of-the-box text metrics for Hugging Face LLMs using LangKit and monitor them in the WhyLabs Observability Platform.

LangKit can extract relevant signals from unstructured text data, such as:

- [Text Quality](https://github.com/whylabs/langkit/blob/main/langkit/docs/features/quality.md)
- [Text Relevance](https://github.com/whylabs/langkit/blob/main/langkit/docs/features/relevance.md)
- [Security and Privacy](https://github.com/whylabs/langkit/blob/main/langkit/docs/features/security.md)
- [Sentiment and Toxicity](https://github.com/whylabs/langkit/blob/main/langkit/docs/features/sentiment.md)

For this example, we'll use the GPT2 model since it's lightweight and easy to run without a GPU, but the example can be run any of the larger Hugging Face models.

![](https://github.com/whylabs/langkit/blob/main/static/img/LangKit_graphic.png?raw=true)


## Setup

To run this notebook
- Google Account `file > save a copy in drive`
- [Free WhyLabs Account](https://whylabs.ai/free)

Other useful links:
- LangKit [GitHub](https://github.com/whylabs/langkit)
- whylogs [GitHub](https://github.com/whylabs/whylogs/)
- [Slack channel](https://bit.ly/r2ai-slack) (Ask questions after the workshop here)





In [ ]:
# Run code cells by pressing the play button
# or hitting Shift+Enter when highlighted
print("Hello, World!")

Hello, World!


### Install Hugging Face Transformers & LangKit

In [ ]:
%pip install transformers
%pip install 'langkit[all]'

## 👋 Hello, World! Take a quick look at LangKit metrics

In the below code we log a few example prompt/response pairs and send metrics to WhyLabs.



In [ ]:
from langkit import llm_metrics # alternatively use 'light_metrics'
import whylogs as why

why.init(session_type='whylabs_anonymous')
# Note: llm_metrics.init() downloads models so this is slow first time.
schema = llm_metrics.init()

In [ ]:
from langkit.whylogs.samples import load_chats, show_first_chat

# Let's look at what's in this toy example:
chats = load_chats()
print(f"There are {len(chats)} records in this toy example data, here's the first one:")
show_first_chat(chats)

results = why.log(chats, name="langkit-sample-chats-all", schema=schema)

##🤗 Use LangKit to monitor LLMs with any Hugging Face model

Import and ititialize the Hugging Face GPT2 model + tokenizer

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
# # Example of loading different models
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-70b-chat-hf")
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-70b-chat-hf")

### Create GPT model function
This will take in a prompt and return a dictionary containing the model response and prompt.

In [ ]:
def gpt_model(prompt):

  # Encode the prompt
  input_ids = tokenizer.encode(prompt, return_tensors='pt')

  # Generate a response
  output = model.generate(input_ids, max_length=100, temperature=0.8,
                          do_sample=True, pad_token_id=tokenizer.eos_token_id)

  # Decode the output
  response = tokenizer.decode(output[0], skip_special_tokens=True)

  # Combine the prompt and the output into a dictionary
  prompt_and_response = {
      "prompt": prompt,
      "response": response
  }

  # print(response)
  return prompt_and_response

In [ ]:
prompt_and_response = gpt_model("Tell me a story about a cute dog")
print(prompt_and_response)

{'prompt': 'Tell me a story about a cute dog', 'response': "Tell me a story about a cute dog that came to my rescue. Where do you get these cute dogs?\n\nI'm here to show you why dogs do good. I think a good dog is pretty much what they're looking for. And you don't have to be a veterinarian to see just how cute they are.\n\nHere are ten reasons why dogs don't turn up to rescue sites.\n\nDo I need to get a dog first?\n\nNo. You just need"}


### Create & Inspect Language Metrics with LangKit

LangKit provides a toolkit of metrics for LLM applications, lets initialize them and create a profile of the data that can be viewed in WhyLabs for quick analysis.

In [ ]:
from langkit import llm_metrics # alternatively use 'light_metrics'
import whylogs as why
import pandas as pd

# Set to show all columns in dataframe
pd.set_option("display.max_columns", None)

# Note: llm_metrics.init() downloads models so this is slow first time.
schema = llm_metrics.init()

In [ ]:
profile = why.log(prompt_and_response, schema=schema).profile()

Skipping uploading profile to WhyLabs because no name was given with name=


We can also see all our values by viewing our LangKit profile in a pandas dataframe.

You can use this data in real-time to make descsion about prompts and reponses, such as setting guardrails on your model.

In [ ]:
profview = profile.view()
profview.to_pandas()

,cardinality/est,cardinality/lower_1,cardinality/upper_1,counts/inf,counts/n,counts/nan,counts/null,distribution/max,distribution/mean,distribution/median,distribution/min,distribution/n,distribution/q_01,distribution/q_05,distribution/q_10,distribution/q_25,distribution/q_75,distribution/q_90,distribution/q_95,distribution/q_99,distribution/stddev,type,types/boolean,types/fractional,types/integral,types/object,types/string,types/tensor,udf/aggregate_reading_level:cardinality/est,udf/aggregate_reading_level:cardinality/lower_1,udf/aggregate_reading_level:cardinality/upper_1,udf/aggregate_reading_level:counts/inf,udf/aggregate_reading_level:counts/n,udf/aggregate_reading_level:counts/nan,udf/aggregate_reading_level:counts/null,udf/aggregate_reading_level:distribution/max,udf/aggregate_reading_level:distribution/mean,udf/aggregate_reading_level:distribution/median,udf/aggregate_reading_level:distribution/min,udf/aggregate_reading_level:distribution/n,udf/aggregate_reading_level:distribution/q_01,udf/aggregate_reading_level:distribution/q_05,udf/aggregate_reading_level:distribution/q_10,udf/aggregate_reading_level:distribution/q_25,udf/aggregate_reading_level:distribution/q_75,udf/aggregate_reading_level:distribution/q_90,udf/aggregate_reading_level:distribution/q_95,udf/aggregate_reading_level:distribution/q_99,udf/aggregate_reading_level:distribution/stddev,udf/aggregate_reading_level:frequent_items/frequent_strings,udf/aggregate_reading_level:types/boolean,udf/aggregate_reading_level:types/fractional,udf/aggregate_reading_level:types/integral,udf/aggregate_reading_level:types/object,udf/aggregate_reading_level:types/string,udf/aggregate_reading_level:types/tensor,udf/automated_readability_index:cardinality/est,udf/automated_readability_index:cardinality/lower_1,udf/automated_readability_index:cardinality/upper_1,udf/automated_readability_index:counts/inf,udf/automated_readability_index:counts/n,udf/automated_readability_index:counts/nan,udf/automated_readability_index:counts/null,udf/automated_readability_index:distribution/max,udf/automated_readability_index:distribution/mean,udf/automated_readability_index:distribution/median,udf/automated_readability_index:distribution/min,udf/automated_readability_index:distribution/n,udf/automated_readability_index:distribution/q_01,udf/automated_readability_index:distribution/q_05,udf/automated_readability_index:distribution/q_10,udf/automated_readability_index:distribution/q_25,udf/automated_readability_index:distribution/q_75,udf/automated_readability_index:distribution/q_90,udf/automated_readability_index:distribution/q_95,udf/automated_readability_index:distribution/q_99,udf/automated_readability_index:distribution/stddev,udf/automated_readability_index:frequent_items/frequent_strings,udf/automated_readability_index:types/boolean,udf/automated_readability_index:types/fractional,udf/automated_readability_index:types/integral,udf/automated_readability_index:types/object,udf/automated_readability_index:types/string,udf/automated_readability_index:types/tensor,udf/character_count:cardinality/est,udf/character_count:cardinality/lower_1,udf/character_count:cardinality/upper_1,udf/character_count:counts/inf,udf/character_count:counts/n,udf/character_count:counts/nan,udf/character_count:counts/null,udf/character_count:distribution/max,udf/character_count:distribution/mean,udf/character_count:distribution/median,udf/character_count:distribution/min,udf/character_count:distribution/n,udf/character_count:distribution/q_01,udf/character_count:distribution/q_05,udf/character_count:distribution/q_10,udf/character_count:distribution/q_25,udf/character_count:distribution/q_75,udf/character_count:distribution/q_90,udf/character_count:distribution/q_95,udf/character_count:distribution/q_99,udf/character_count:distribution/stddev,udf/character_count:frequent_items/frequent_strings,udf/character_count:types/boolean,udf/character_count:types/fractional,udf/character_count:types/integral,udf/character_cou

### Muliple Prompts

In [ ]:
prompts = ["What is AI?",
           "Tell me a joke.",
           "Who won the world series in 2021?"]

In [ ]:
for num, prompt in enumerate(prompts):

  prompt_and_response = gpt_model(prompt)

  # initial profile schema on first profile
  if num == 0:
    profile = why.log(prompt_and_response, schema=schema).profile()
  profile.track(prompt_and_response)

Skipping uploading profile to WhyLabs because no name was given with name=


In [ ]:
profview = profile.view()
profview.to_pandas()

,cardinality/est,cardinality/lower_1,cardinality/upper_1,counts/inf,counts/n,counts/nan,counts/null,distribution/max,distribution/mean,distribution/median,distribution/min,distribution/n,distribution/q_01,distribution/q_05,distribution/q_10,distribution/q_25,distribution/q_75,distribution/q_90,distribution/q_95,distribution/q_99,distribution/stddev,type,types/boolean,types/fractional,types/integral,types/object,types/string,types/tensor,udf/aggregate_reading_level:cardinality/est,udf/aggregate_reading_level:cardinality/lower_1,udf/aggregate_reading_level:cardinality/upper_1,udf/aggregate_reading_level:counts/inf,udf/aggregate_reading_level:counts/n,udf/aggregate_reading_level:counts/nan,udf/aggregate_reading_level:counts/null,udf/aggregate_reading_level:distribution/max,udf/aggregate_reading_level:distribution/mean,udf/aggregate_reading_level:distribution/median,udf/aggregate_reading_level:distribution/min,udf/aggregate_reading_level:distribution/n,udf/aggregate_reading_level:distribution/q_01,udf/aggregate_reading_level:distribution/q_05,udf/aggregate_reading_level:distribution/q_10,udf/aggregate_reading_level:distribution/q_25,udf/aggregate_reading_level:distribution/q_75,udf/aggregate_reading_level:distribution/q_90,udf/aggregate_reading_level:distribution/q_95,udf/aggregate_reading_level:distribution/q_99,udf/aggregate_reading_level:distribution/stddev,udf/aggregate_reading_level:frequent_items/frequent_strings,udf/aggregate_reading_level:types/boolean,udf/aggregate_reading_level:types/fractional,udf/aggregate_reading_level:types/integral,udf/aggregate_reading_level:types/object,udf/aggregate_reading_level:types/string,udf/aggregate_reading_level:types/tensor,udf/automated_readability_index:cardinality/est,udf/automated_readability_index:cardinality/lower_1,udf/automated_readability_index:cardinality/upper_1,udf/automated_readability_index:counts/inf,udf/automated_readability_index:counts/n,udf/automated_readability_index:counts/nan,udf/automated_readability_index:counts/null,udf/automated_readability_index:distribution/max,udf/automated_readability_index:distribution/mean,udf/automated_readability_index:distribution/median,udf/automated_readability_index:distribution/min,udf/automated_readability_index:distribution/n,udf/automated_readability_index:distribution/q_01,udf/automated_readability_index:distribution/q_05,udf/automated_readability_index:distribution/q_10,udf/automated_readability_index:distribution/q_25,udf/automated_readability_index:distribution/q_75,udf/automated_readability_index:distribution/q_90,udf/automated_readability_index:distribution/q_95,udf/automated_readability_index:distribution/q_99,udf/automated_readability_index:distribution/stddev,udf/automated_readability_index:frequent_items/frequent_strings,udf/automated_readability_index:types/boolean,udf/automated_readability_index:types/fractional,udf/automated_readability_index:types/integral,udf/automated_readability_index:types/object,udf/automated_readability_index:types/string,udf/automated_readability_index:types/tensor,udf/character_count:cardinality/est,udf/character_count:cardinality/lower_1,udf/character_count:cardinality/upper_1,udf/character_count:counts/inf,udf/character_count:counts/n,udf/character_count:counts/nan,udf/character_count:counts/null,udf/character_count:distribution/max,udf/character_count:distribution/mean,udf/character_count:distribution/median,udf/character_count:distribution/min,udf/character_count:distribution/n,udf/character_count:distribution/q_01,udf/character_count:distribution/q_05,udf/character_count:distribution/q_10,udf/character_count:distribution/q_25,udf/character_count:distribution/q_75,udf/character_count:distribution/q_90,udf/character_count:distribution/q_95,udf/character_count:distribution/q_99,udf/character_count:distribution/stddev,udf/character_count:frequent_items/frequent_strings,udf/character_count:types/boolean,udf/character_count:types/fractional,udf/character_count:types/integral,udf/character_cou

Having the distribution values are important for ML monitoring

![](https://raw.githubusercontent.com/whylabs/langkit/dbc11994e094a3ade6425bdc0506cecfee724f7d/static/img/sentiment-monitor.png)

##👀 ML Monitoring for Hugging Face LLMs in WhyLabs


To send LangKit profiles to WhyLabs we will need three pieces of information:

- API token
- Organization ID
- Dataset ID (or model-id)

Go to [https://whylabs.ai/free](https://whylabs.ai/free) and grab a free account. You can follow along with the quick start examples or skip them if you'd like to follow this example immediately.

1. Create a new project and note its ID (if it's a model project, it will look like `model-xxxx`)
2. Create an API token from the "Access Tokens" tab
3. Copy your org ID from the same "Access Tokens" tab

Replace the placeholder string values with your own OpenAI and WhyLabs API Keys below:

In [ ]:
import os
# set authentication & project keys
os.environ["WHYLABS_DEFAULT_ORG_ID"] = 'ORGID'
os.environ["WHYLABS_API_KEY"] = 'APIKEY'
os.environ["WHYLABS_DEFAULT_DATASET_ID"] = 'MODELID'

In [ ]:
from whylogs.api.writer.whylabs import WhyLabsWriter
from langkit import llm_metrics # alternatively use 'light_metrics'
import whylogs as why

# Note: llm_metrics.init() downloads models so this is slow first time.
schema = llm_metrics.init()

In [ ]:
# Single Profile
telemetry_agent = WhyLabsWriter()
profile = why.log(prompt_and_response, schema=schema)
telemetry_agent.write(profile.view())

Skipping uploading profile to WhyLabs because no name was given with name=


(True, 'log-FDc6cM0M6sz70n2X')

This will write a single profile to WhyLabs

Note: you may see `Skipping uploading profile to WhyLabs because no name was given with name=` ignore for now. This message won't appear if you do not use the whylabs_anonymous session first!


### Back Filling

Write seven day prompt list








In [ ]:
prompt_lists = [
    ["How can I create a new account?", "Great job to the team", "Fantastic product, had a good experience"],
    ["This product made me angry, can I return it", "You dumb and smell bad", "I hated the experience, and I was over charged"],
    ["This seems amazing, could you share the pricing?", "Incredible site, could we setup a call?", "Hello! Can you kindly guide me through the documentation?"],
    ["This looks impressive, could you provide some information on the cost?", "Stunning platform, can we arrange a chat?", "Hello there! Could you assist me with the documentation?"],
    ["This looks remarkable, could you tell me the price range?", "Fantastic webpage, is it possible to organize a call?", "Greetings! Can you help me with the relevant documents?"],
    ["This is great, Ilove it, could you inform me about the charges?", "love the interface, can we have a teleconference?", "Hello! Can I take a look at the user manuals?"],
    ["This seems fantastic, how much does it cost?", "Excellent website, can we setup a call?", "Hello! Could you help me find the resource documents?"]
]


In [ ]:
import datetime

In [ ]:

telemetry_agent = WhyLabsWriter()
all_prompts_and_responses = []  # This list will store all the prompts and responses.


for i, day in enumerate(prompt_lists):
  # walking backwards. Each dataset has to map to a date to show up as a different batch in WhyLabs
  dt = datetime.datetime.now(tz=datetime.timezone.utc) - datetime.timedelta(days=i)
  for prompt in day:
    prompt_and_response = gpt_model(prompt)
    profile = why.log(prompt_and_response, schema=schema)

     # Save the prompt and its response in the list.
    all_prompts_and_responses.append({'prompt': prompt, 'response': prompt_and_response})

    # set the dataset timestamp for the profile
    profile.set_dataset_timestamp(dt)
    telemetry_agent.write(profile.view())

In [ ]:
all_prompts_and_responses

### Guardrails & Validation in environment



In [ ]:
import os
import whylogs as why
import langkit.toxicity
import pandas as pd

from langkit import llm_metrics

print("downloading models and initialized metrics...")
text_metrics_schema = llm_metrics.init()

# Set to show all columns in dataframe
pd.set_option("display.max_columns", None)


downloading models and initialized metrics...


In [ ]:
print("downloading models and initialized metrics...")
text_metrics_schema = llm_metrics.init()


downloading models and initialized metrics...


In [ ]:
def is_not_toxic(prompt_message):
    # Create profile of prompt
    profile = why.log({"prompt": prompt_message}, schema=text_metrics_schema).profile().view()

    profile_dict = profile.get_column('prompt').to_summary_dict()
    tox_max = profile_dict['udf/toxicity:distribution/max']

    print(f'Toxic score: {tox_max}')

    if tox_max > 0.5:
      return False
    else:
      return True

In [ ]:
is_not_toxic("Do you like fruit?")

Skipping uploading profile to WhyLabs because no name was given with name=
Toxic score: 0.0014600753784179688


True

In [ ]:
is_not_toxic("You dumb and smell bad")

Skipping uploading profile to WhyLabs because no name was given with name=
Toxic score: 0.9606605768203735


False

In [ ]:
user_prompt = "Do you like fruit?"

if is_not_toxic(user_prompt):
  prompt_and_response = gpt_model(user_prompt)
  print(prompt_and_response['response'])

else:
  print("As a large language model...")

In [ ]:
user_prompt = "You dumb and smell bad"

if is_not_toxic(user_prompt):
  prompt_and_response = gpt_model(user_prompt)
  print(prompt_and_response['response'])

else:
  print("As a large language model...")

See another way of doing this with [LangKit validators](https://whylabs.ai/blog/posts/safeguard-monitor-large-language-model-llm-applications)

## Use a Rolling Logger
A rolling logger can be used instead of the method above to write profiles to WhyLabs at pre-defined intervals.

In [ ]:
telemetry_agent = why.logger(mode="rolling", interval=5, when="M",schema=schema, base_name="huggingface")
telemetry_agent.append_writer("whylabs")

In [ ]:
# Log data + model outputs to WhyLabs.ai
telemetry_agent.log(prompt_and_response)

In [ ]:
# Close the whylogs rolling logger when the service is shut down
telemetry_agent.close()

# Resources

- [Intro to LangKit Example](https://github.com/whylabs/langkit/blob/main/langkit/examples/Intro_to_Langkit.ipynb)
- [LangKit + LangChain Integration](https://github.com/whylabs/langkit/blob/main/langkit/examples/Langchain_OpenAI_LLM_Monitoring_with_WhyLabs.ipynb)
- [LangKit GitHub](https://github.com/whylabs/langkit)
- [whylogs GitHub](https://github.com/whylabs/whylogs)
- [WhyLabs](https://whylabs.ai/safeguard-large-language-models)
- [Hugging Face GPT2 Model](https://huggingface.co/gpt2)